<a href="https://colab.research.google.com/github/OvoFiona/AI-COURSEWORK-2025-/blob/main/supply_chain_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from google.colab import files
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Load the dataset from the uploaded CSV file
print("Please upload the 'supply_chain_data.csv' file:")
uploaded = files.upload()
filename = list(uploaded.keys())[0]
df = pd.read_csv(filename)

Please upload the 'supply_chain_data.csv' file:


Saving supply_chain_data.csv to supply_chain_data.csv


In [2]:
#initial exploration
print("Dataset shape: ", df.shape)
print("\nData Types:\n ", df.dtypes)
print("\nMissing Values:\n", df.isnull().sum())

Dataset shape:  (100, 24)

Data Types:
  Product type                object
SKU                         object
Price                      float64
Availability                 int64
Number of products sold      int64
Revenue generated          float64
Customer demographics       object
Stock levels                 int64
Lead times                   int64
Order quantities             int64
Shipping times               int64
Shipping carriers           object
Shipping costs             float64
Supplier name               object
Location                    object
Lead time                    int64
Production volumes           int64
Manufacturing lead time      int64
Manufacturing costs        float64
Inspection results          object
Defect rates               float64
Transportation modes        object
Routes                      object
Costs                      float64
dtype: object

Missing Values:
 Product type               0
SKU                        0
Price                      0


In [21]:
#Data Cleaning
#Check for missing values
print("\nChecking for missing values:")
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])

#Check for duplicates
print(f"\nDuplicate rows: {df.duplicated().sum()}")

#Remove duplicated rows if any
supply_chain_data_df = df.drop_duplicates()

#Check for inconsistent values in categorical columns
print("\nChecking categorical columns for inconsistencies:")

categorical_cols = ['Product type', 'SKU', 'Availability', 'Customer demographics',
                   'Shipping carriers', 'Supplier name', 'Location', 'Inspection results',
                   'Transportation modes', 'Routes']

for col in categorical_cols:
    if col in df.columns:
        print(f"\n{col}:")
        print(f"Unique values: {df[col].nunique()}")
        print(f"Sample values: {df[col].unique()[:5]}")

#Check for outliers in numerical columns
print("\nStatistical summary for numerical columns:")
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print(df[numerical_cols].describe())

#Check for inconsistent column names (case sensitivity, spaces)
print("\nColumn names (checking for inconsistencies):")
print(df.columns)

#Check for invalid values in specific columns
print("\nChecking for invalid values:")

# Check Price (should be positive)
negative_prices = df[df['Price'] < 0].shape[0]
print(f"Negative prices: {negative_prices}")

# Check Number of products sold (should be non-negative)
negative_sales = df[df['Number of products sold'] < 0].shape[0]
print(f"Negative sales: {negative_sales}")

# Check Stock levels (should be non-negative)
negative_stock = df[df['Stock levels'] < 0].shape[0]
print(f"Negative stock levels: {negative_stock}")

# Check Defect rates (should be between 0 and 100)
invalid_defects = df[(df['Defect rates'] < 0) | (df['Defect rates'] > 100)].shape[0]
print(f"Invalid defect rates: {invalid_defects}")

#Check for duplicate column names or similar columns
print("\nChecking for similar/duplicate columns:")
# Note: There are two 'Lead time' columns - one from supplier and one from manufacturing
lead_time_cols = [col for col in df.columns if 'lead' in col.lower()]
print(f"Lead time related columns: {lead_time_cols}")

#Create a copy for cleaning
df_clean = df.copy()

#Rename columns for clarity
print("\nRenaming columns for clarity...")
df_clean = df_clean.rename(columns={
    'Lead time': 'Supplier lead time',  # This is the supplier lead time
    'Lead times': 'Order processing time'  # This appears to be order processing time
})

#Fix data type issues
print("\nChecking and fixing data types...")
# Convert appropriate columns to integer
int_columns = ['Number of products sold', 'Stock levels', 'Order quantities',
               'Supplier lead time', 'Order processing time', 'Shipping times',
               'Production volumes', 'Manufacturing lead time']

for col in int_columns:
    if col in df_clean.columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce').astype('Int64')
        print(f"  - Converted '{col}' to integer type")

#Handle outliers (using IQR method)
print("\nHandling outliers in key numerical columns...")
key_numerical = ['Price', 'Number of products sold', 'Revenue generated',
                'Defect rates', 'Shipping costs', 'Costs', 'Manufacturing costs']

for col in key_numerical:
    if col in df_clean.columns:
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        outliers = df_clean[(df_clean[col] < lower_bound) | (df_clean[col] > upper_bound)].shape[0]
        if outliers > 0:
            # Cap the outliers instead of removing them
            df_clean[col] = np.where(df_clean[col] < lower_bound, lower_bound, df_clean[col])
            df_clean[col] = np.where(df_clean[col] > upper_bound, upper_bound, df_clean[col])
            print(f"  - Capped {outliers} outliers in '{col}' using IQR method")


#Standardize categorical values
print("\nStandardizing categorical values...")

# Standardize text case
text_columns = ['Product type', 'Customer demographics', 'Shipping carriers',
               'Supplier name', 'Location', 'Inspection results',
               'Transportation modes', 'Routes']

for col in text_columns:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].astype(str).str.title().str.strip()
        print(f"  - Standardized case and trimmed whitespace in '{col}'")

#Validate and correct specific fields
print("\nValidating specific fields...")

# Ensure defect rates are between 0 and 100
if 'Defect rates' in df_clean.columns:
    df_clean['Defect rates'] = df_clean['Defect rates'].clip(lower=0, upper=100)
    print("  - Clipped defect rates to 0-100% range")

# Ensure positive values for price and quantities
positive_columns = ['Price', 'Number of products sold', 'Revenue generated',
                   'Stock levels', 'Order quantities']

for col in positive_columns:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].clip(lower=0)
        print(f"  - Ensured non-negative values in '{col}'")

#Create new calculated columns if needed
print("\nCreating calculated columns...")

# Calculate profit margin if manufacturing cost is available
if all(col in df_clean.columns for col in ['Price', 'Manufacturing costs']):
    df_clean['Profit_margin_per_unit'] = df_clean['Price'] - df_clean['Manufacturing costs']
    df_clean['Profit_margin_percentage'] = ((df_clean['Price'] - df_clean['Manufacturing costs']) / df_clean['Price'] * 100).round(2)
    print("  - Created 'Profit_margin_per_unit' and 'Profit_margin_percentage' columns")

# Calculate total lead time
if all(col in df_clean.columns for col in ['Supplier lead time', 'Manufacturing lead time', 'Order processing time']):
    df_clean['Total_lead_time'] = df_clean['Supplier lead time'] + df_clean['Manufacturing lead time'] + df_clean['Order processing time']
    print("  - Created 'Total_lead_time' column")

# Calculate inventory turnover ratio
if all(col in df_clean.columns for col in ['Number of products sold', 'Stock levels']):
    df_clean['Inventory_turnover_ratio'] = np.where(
        df_clean['Stock levels'] > 0,
        df_clean['Number of products sold'] / df_clean['Stock levels'],
        0
    ).round(2)
    print("  - Created 'Inventory_turnover_ratio' column")
#Final validation
print("\n" + "=" * 50)
print("FINAL DATASET VALIDATION")
print("=" * 50)

print(f"\nCleaned dataset shape: {df_clean.shape}")
print(f"\nMissing values after cleaning:")
print(df_clean.isnull().sum())

print(f"\nData types after cleaning:")
print(df_clean.dtypes)

# Check for any remaining issues
print("\nRemaining potential issues:")

# Check for negative values in calculated columns
if 'Profit_margin_per_unit' in df_clean.columns:
    negative_margin = df_clean[df_clean['Profit_margin_per_unit'] < 0].shape[0]
    print(f"  - Products with negative profit margin: {negative_margin}")

# Check for unrealistic defect rates
if 'Defect rates' in df_clean.columns:
    high_defects = df_clean[df_clean['Defect rates'] > 10].shape[0]
    print(f"  - Products with defect rates >10%: {high_defects}")

# Save cleaned dataset
output_file = 'supply_chain_data_cleaned.csv'
df_clean.to_csv(output_file, index=False)
print(f"\n✅ Data cleaning completed!")
print(f"✅ Cleaned dataset saved to: {output_file}")

# Display sample of cleaned data
print(f"\nSample of cleaned data (first 5 rows):")
print(df_clean.head())




Checking for missing values:
Series([], dtype: int64)

Duplicate rows: 0

Checking categorical columns for inconsistencies:

Product type:
Unique values: 3
Sample values: ['haircare' 'skincare' 'cosmetics']

SKU:
Unique values: 100
Sample values: ['SKU0' 'SKU1' 'SKU2' 'SKU3' 'SKU4']

Availability:
Unique values: 63
Sample values: [55 95 34 68 26]

Customer demographics:
Unique values: 4
Sample values: ['Non-binary' 'Female' 'Unknown' 'Male']

Shipping carriers:
Unique values: 3
Sample values: ['Carrier B' 'Carrier A' 'Carrier C']

Supplier name:
Unique values: 5
Sample values: ['Supplier 3' 'Supplier 1' 'Supplier 5' 'Supplier 4' 'Supplier 2']

Location:
Unique values: 5
Sample values: ['Mumbai' 'Kolkata' 'Delhi' 'Bangalore' 'Chennai']

Inspection results:
Unique values: 3
Sample values: ['Pending' 'Fail' 'Pass']

Transportation modes:
Unique values: 4
Sample values: ['Road' 'Air' 'Rail' 'Sea']

Routes:
Unique values: 3
Sample values: ['Route B' 'Route C' 'Route A']

Statistical summar